In [1]:
# for more in depth simulations, see Skelet's BBprover program

import copy
from collections import defaultdict
import time

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


class TuringMachine:
    def __init__(self, delta, q0, F, max_moves, time, save, custom_tape):
        """
            Keyword Arguments:
            delta                  -- transition function used for all TMs
            q0                     -- starting state of TM
            F                      -- halting states. Only 1 state needed for BB(n).
            max_moves              -- avoid looping forever. This is the bottom-up approach
            time                   -- speed testing
            save                   -- record state of tape for analysis
            custom_tape            -- inputs for custom tape analysis
        """

        # self.states = Q
        # self.tape_symbs = Gamma
        # self.blank = b
        # self.input_symbs = Sigma
        self.step = delta       
        self.initial = q0       
        self.halts = F          
        self.max_moves = max_moves           
        self.position = 0
        self.moves = 0
        self.ones = 0
        self.start_time = time
        self.candidates = []
        self.tape_save = save
        self.tape_custom = custom_tape

    def run_tm(self):
        # Run the machine

        # Use defaultdict to key in cell positions relative to starting position
        state = self.initial
        tape = defaultdict(
            lambda: 0
        )  

        # Impliment custom inputs
        if self.tape_custom != 0:
            for key, value in self.tape_custom.items():
                tape[key] = value

        # Run transition function on tape
        while (state != self.halts) and (self.moves < self.max_moves):
            write, shift, state = self.step[state][tape[self.position]]
            tape[self.position] = int(write)
            self.position += 1 if shift == "r" else -1
            self.moves += 1

        # Count output one's
        for output in tape.values():
            self.ones += int(output)

        # Saves tape for custom run
        if self.tape_save == 1: 
            self.tape_save = [tape]


class Busy_beaver: 
    def __init__(self, n, max_run, time):
        """Run BBn algorithms

            Keyword Arguments:
            n                      -- runs only n-state turing machines
            max_run                -- states
            time                   -- speed testing
        """

        self.start_time = time  
        self.n = n
        self.stop_step = max_run
        self.moves = []
        self.ones = []
        self.tm = []
        self.cards = []
        self.library = {
            1: ("0", "1"),
            2: ("r", "l"),
            3: ["_", "a", "b", "c", "d", "e", "f"],
        }
        self.library[3] = self.library[3][0 : n + 1]

        # template -> {state:{value:(write, move right/left, change state}}
        self.card_template = [  
                "",
                {"a": {0: "", 1: ""}},
                {"a": {0: "", 1: ""}, "b": {0: "", 1: ""}},  # BB(2)
                {"a": {0: "", 1: ""}, "b": {0: "", 1: ""}, "c": {0: "", 1: ""}},
                {
                "a": {0: (""), 1: ""},
                "b": {0: "", 1: ""},
                "c": {0: "", 1: ""},
                "d": {0: "", 1: ""},
                },
                {
                "a": {0: (""), 1: ""},
                "b": {0: "", 1: ""},
                "c": {0: "", 1: ""},
                "d": {0: "", 1: ""},
                "e": {0: "", 1: ""},
                },
                {
                "a": {0: (""), 1: ""},
                "b": {0: "", 1: ""},
                "c": {0: "", 1: ""},
                "d": {0: "", 1: ""},
                "e": {0: "", 1: ""},
                "f": {0: "", 1: ""},
                },
            ]
        # Corresponding Turing Machines for BBn ordered numerically 1-5
        self.initialize_bbn = [
            "1r_1r_",  
            "1rb1lb1la1r_", 
            "1rb1r_0rc1rb1lc1la",
            "1rb1lb1la0lc1r_1ld1rd0ra",
            "1rb1lc1rc1rb1rd0le1la1ld1r_0la"      
            # this ~4 million step run is only known lower bound found for 5
            # or use '1lb1la1rc1rb1la1rd1la1re1r_0rc' for speedup
            # ~1 million step 5 state 4098 TM same lower bound for 1's printed
            ]

    def build(self, current, max_len): 
        '''Builds all possible n-state transition functions recursively
            
            Keyword Arguments:
            current                -- current string (transition function) being built
            max_len                -- set state count for all machines
        '''

        if len(current) == max_len:
            return self.cards.append(current[1:])
        elif current[-1] in self.library[1]:  # these need speedup through len algebra
            for i in self.library[2]:
                self.build(current + i, max_len)
        elif current[-1] in self.library[2]:
            if (len(current) == (max_len - 1)) and ("_" not in current):
                current += "_"
                return self.cards.append(current[1:])
            else:
                for i in self.library[3]:
                    self.build(current + i, max_len)
        else:
            for i in self.library[1]:
                self.build(current + i, max_len)

    def card_maker(self, cards, initializer, n):   
        '''Convert cards into nice format for our TM class
            
            Keyword Arguments:
            cards                -- transition function cards represented as strings
            initializer          -- the conversion table for selected state n
        '''

        self.tm = []
        states = self.library[3]

        # Check for custom tape input. Write each card into initializer clone. 
        if type(cards) is list:
            for card in cards:
                temp = copy.deepcopy(initializer)
                for count in range(1, n + 1):
                    temp[states[count]][0] = tuple(
                        card[count * 6 - 6 : count * 6 - 3]
                        )
                    temp[states[count]][1] = tuple(
                        card[count * 6 - 3 : count * 6]
                        )
                self.tm.append(temp.copy())
        else:
            for count in range(1, n + 1):
                initializer[states[count]][0] = tuple(
                    cards[count * 6 - 6 : count * 6 - 3]
                    )
                initializer[states[count]][1] = tuple(
                    cards[count * 6 - 3 : count * 6]
                    )
            self.tm = initializer

    def run_sim(self):
        # Run BBn simulation

        selection = self.initialize_bbn[self.n - 1]
        self.card_maker(selection, self.card_template[self.n], self.n)
        bbn = TuringMachine(
            self.tm, ("a"), ("_"), self.stop_step, self.start_time, 0, 0
        )
        bbns.run_tm()
        s1, s2 = (bbn.moves > 1) * "s", (bbn.ones > 1) * "s"
        print(f"BB{self.n} finished with {bbn.moves} step{s1} \
              and with {bbn.ones} one{s2} on the tape")

    def run_sim_custom(self, custom, tape):  
        # Run custom simulation for state/tape input pair simulations

        selection = custom
        self.card_maker(selection, self.card_template[self.n], self.n)
        bbn = TuringMachine(
            self.tm, ("a"), ("_"), self.stop_step, self.start_time, 1, tape
        )
        bbn.run_tm()
        print(bbn.tape_custom)
        s1, s2 = (bbn.moves > 1) * "s", (bbn.ones > 1) * "s"
        print(f"BB{self.n} finished with {bbn.moves} step{s1} \
              and with {bbn.ones} one{s2} on the tape")

    def run_find(self):  
        # Run all candidate TMs for analysis

        # Build all machines and remove reduntant halters
        if self.n > 1:
            self.build("-", (self.n) * 6 - 2)
            temp_list = list(
                filter(lambda x: "_" not in x.replace("_", "", 1), self.cards)
            )  
            self.cards = ["1rb" + x for x in temp_list]
        else:
            self.cards = ["1r_1r_"]
        self.card_maker(self.cards, self.card_template[self.n], self.n)

        #display current process
        print("running " + str(len(self.cards)) + f" candidates for BB({self.n}, 2)")
        output = {0: 0, 1: defaultdict(lambda: 0), 2: defaultdict(lambda: 0)}

        # Run with time and step counts
        for card in self.tm:
            output[0] += 1

            if output[0] % 100000 == 0:
                print("search completed " + str(output[0]) + " machines")  
            # if round(time.time() - self.start_time, 1) > 30:
            #     sys.exit("tmrunner needs optimizing")

            bbn = TuringMachine(
                card, ("a"), ("_"), self.stop_step, self.start_time, 0, 0
            )
            bbn.run_tm()
            output[1][bbn.moves] += 1

            if bbn.moves < self.stop_step:
                output[2][bbn.ones] += 1

        # plot the output
        m1, m2 = zip(*sorted(output[1].items(), key=lambda t: t[0]))
        m1, m2 = list(m1), list(m2)
        if self.n > 1:
            index_var = m1.index(max(m1))
            m1[index_] = "n/a"
            temp = m2[index_var]
            m2[index_var] = -1
        o1, o2 = zip(*sorted(output[2].items(), key=lambda t: t[0]))
        o1 = list(o1)
        k = (12000) ** (self.n-2) + 40

        fig, axs = plt.subplots(2, 1)
        fig.suptitle(f"BB({self.n}) runtime analysis")
        axs[0].bar([f"{x}" for x in m1], m2)
        axs[1].bar([f"{x}" for x in o1], o2)
        axs[0].set_xlabel("Moves Before Halt")
        axs[1].set_xlabel("Ones Before Halt")
        for i in range(2):
            axs[i].set_ylim((0, 2 * max(m2)))
            axs[i].set_ylabel("Number of TMs")
            axs[i].xaxis.label.set_color("darkblue")
            axs[i].yaxis.label.set_color("darkblue")
            axs[i].grid(True)
        if self.n > 1:
            for i, v in enumerate(m2):
                if v == -1:
                    v = 0
                    m2[i] = temp
                axs[0].text(
                    i - 0.1,
                    v + k,
                    m2[i],
                    fontsize=11,
                    color="maroon",
                    fontweight="bold",
                    rotation=45,
                )
            for i, v in enumerate(o2):
                axs[1].text(
                    i - 0.1,
                    v + k,
                    o2[i],
                    fontsize=11,
                    color="maroon",
                    fontweight="bold",
                    rotation=45,
                )

        plt.tight_layout()
        plt.show()

In [ ]:
def launch_bbn_finder():
    """The machine search method searches for BB(1-3). 
    It uses a bottom up approach to find candidates, 
    graphing out all active machines into their 
    respective runtime and output catagories. 
    Prelimanary checks for halt counts and other 
    redundancies are implimented to reduce the 
    necessary test cases.
    """

    print("Filtering for busy beaver candidates (states 1-3)")
    for n in range(1, 4):
        start_time = time.time()
        bb = Busy_beaver(n, 10*(n-1) + 2, start_time)
        bb.run_find()
        print(f"time taken: {round(time.time() - start_time, 1)}s")
        print("")
        
launch_bbn_finder()

In [ ]:
def run_custom(): 
    # for custom case testing 

    start_time = time.time()
    bbn = Busy_beaver(6, 3838, start_time)
    bbn.run_sim_custom("1rb1le1rc1rf1ld0rb1re0lc1la0rd1r_1rc", {0:0, 1:1, 2:1}) 
        # currently, this is confirming section 3.5 step 2 from the proof of 
        # Wythagoras' machine from the paper (linked in the readme).
        
run_custom()

In [ ]:
def launch_bbn_ones():
    # run lower bound on BBn 1-5

    print("This program runs BB1, BB2, BB3, BB4, and the known \
        BB5 lower bound defined by the amount of 1\'s printed to the tape")

    max_step = [5, 10, 25, 110, 50000000]
    for n in range(1, 6):
        start_time = time.time()
        bbn = Busy_beaver(n, max_step[n - 1], start_time)
        bbn.run_sim()
        print(f"time taken: {round(time.time() - start_time, 1)}s")
        print("")

launch_bbn_ones()